In [134]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Derive annual means from monthly data

In [135]:
import xarray as xr
import numpy as np
import gfdl_utils as gu
import glob
from dask.diagnostics import ProgressBar
from information import *
from processing import *
from variance import *
import os

In [127]:
variable = 'intpp'
frequency='monthly'
masks = generate_masks(grid)
saverootdir = ppeDict['datasavedir']+'/raw/annualmeans/'
verbose = False

In [131]:
def calc_annualmeans(paths,saverootdir):
    for path in paths:
        ds = xr.open_dataset(path)
        newds = ds.coarsen(time=12).mean()
        # Get details of new path
        path_cmp = path.split('/')
        newdir_cmp = path_cmp[4:-1]
        newdir_cmp[3]=newppname
        newdir_cmp[-2]='annual'
        newdir = saverootdir+'/'.join(newdir_cmp)
        if not os.path.isdir(newdir):
            os.makedirs(newdir)
        timestr = os.path.basename(path).split('.')[-3]
        newtimestr = timestr[0:4]+'-'+timestr[7:11]
        filename = '.'.join([ppname_new,newtimestr,variable,'nc'])
        newpath = '/'.join([newdir,filename])
        if os.path.exists(newpath):
            print(filename+' exists')
        else:
            print(filename)
            with ProgressBar():
                newds.to_netcdf(newpath)

In [132]:
print('ENSEMBLES')
for startyear in ppeDict['startyears']:
    for startmonth in ppeDict['startmonths']:
        for member in np.arange(1,10):
            print(str(startyear)+', '+str(startmonth)+', '+str(member))
            pp = get_pp(getmember=True,startyear=startyear,startmonth=startmonth,member=member)
            pathDict = get_pathDict_from_pp_variable_frequency(pp,variable,frequency,time='*',out='ts',constraint=None)
            paths = glob.glob(gu.core.get_pathspp(**pathDict))
            calc_annualmeans(paths,saverootdir)
            
print('CONTROL')
pp = get_pp()
pathDict = get_pathDict_from_pp_variable_frequency(pp,variable,frequency,time='*',out='ts',constraint=None)
paths = glob.glob(gu.core.get_pathspp(**pathDict))
calc_annualmeans(paths,saverootdir)

ENSEMBLES
123, 1, 1
ocean_cobalt_omip_2d_annual.0123-0127.intpp.nc exists
ocean_cobalt_omip_2d_annual.0128-0132.intpp.nc exists
123, 1, 2
ocean_cobalt_omip_2d_annual.0123-0127.intpp.nc exists
ocean_cobalt_omip_2d_annual.0128-0132.intpp.nc exists
123, 1, 3
ocean_cobalt_omip_2d_annual.0123-0127.intpp.nc exists
ocean_cobalt_omip_2d_annual.0128-0132.intpp.nc exists
123, 1, 4
ocean_cobalt_omip_2d_annual.0123-0127.intpp.nc
ocean_cobalt_omip_2d_annual.0128-0132.intpp.nc
123, 1, 5
ocean_cobalt_omip_2d_annual.0123-0127.intpp.nc
ocean_cobalt_omip_2d_annual.0128-0132.intpp.nc
123, 1, 6
ocean_cobalt_omip_2d_annual.0123-0127.intpp.nc
ocean_cobalt_omip_2d_annual.0128-0132.intpp.nc
123, 1, 7
ocean_cobalt_omip_2d_annual.0123-0127.intpp.nc
ocean_cobalt_omip_2d_annual.0128-0132.intpp.nc
123, 1, 8
ocean_cobalt_omip_2d_annual.0123-0127.intpp.nc
ocean_cobalt_omip_2d_annual.0128-0132.intpp.nc
123, 1, 9
ocean_cobalt_omip_2d_annual.0123-0127.intpp.nc
ocean_cobalt_omip_2d_annual.0128-0132.intpp.nc
123, 4, 1
oc

In [144]:
ds = open_control(variable,'annual')

In [158]:
tmp = ds[variable].std('time').expand_dims({'time':1})
times = [0]*10
tmp.isel(time=times)

<xarray.DataArray 'intpp' (time: 10, yh: 576, xh: 720)>
dask.array<getitem, shape=(10, 576, 720), dtype=float32, chunksize=(10, 576, 720), chunktype=numpy.ndarray>
Coordinates:
  * xh       (xh) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * yh       (yh) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
Dimensions without coordinates: time

In [159]:
calc_cvar(variable,'annual')

Calculating control variance.


<xarray.Dataset>
Dimensions:  (xh: 720, yh: 576, nv: 2, time: 10)
Coordinates:
  * xh       (xh) float64 -299.8 -299.2 -298.8 -298.2 ... 58.75 59.25 59.75
  * yh       (yh) float64 -77.91 -77.72 -77.54 -77.36 ... 89.47 89.68 89.89
  * nv       (nv) float64 1.0 2.0
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    intpp    (time, yh, xh) float32 dask.array<chunksize=(1, 576, 720), meta=np.ndarray>

In [142]:
rootdir = ppeDict['rootdir_annual']
get_pp(rootdir=rootdir)

'/work/gam/projects/esm4_ppe/data/raw/annualmeans/ESM4_piControl_D/gfdl.ncrc4-intel18-prod-openmp/pp'

In [141]:
ppeDict['rootdir_annual']

'/work/gam/projects/esm4_ppe/data/raw/annualmeans'